In [6]:
#### Script for training

In [7]:
import re
import sqlite3
import os
import csv
import pandas as pd
from tqdm.auto import tqdm

counterTableName = 'counter'
graphTableName = 'graph'
edgesTableName = 'edges'

timeStart = 0
graphId = 0

In [8]:
def prepare():
    conn = sqlite3.connect(r'toy.db')
    cur = conn.cursor()

    qry = """ SELECT * from {} """.format(counterTableName)
    time = cur.execute(qry).fetchone()
    
    qry = """ SELECT * from {} """.format(graphTableName)
    graph_id = cur.execute(qry).fetchone()
    
    if time == None:
        qry = """ insert into {} values (0) """.format(counterTableName)
        cur.execute(qry)
    if graph_id == None:
        qry = """ insert into {} values (0) """.format(graphTableName)
        cur.execute(qry)  
    conn.commit()
    
    qry = """ SELECT * from {} """.format(counterTableName)
    time = cur.execute(qry).fetchone()
    
    qry = """ SELECT * from {} """.format(graphTableName)
    graph_id = cur.execute(qry).fetchone()
    conn.close()
    
    global timeStart 
    timeStart  = time[0]
    global graphId 
    graphId = graph_id[0]
    
    print('time - {}  id - {}'.format( timeStart,  graphId))

def updateDB():
    conn = sqlite3.connect(r'toy.db')
    cur = conn.cursor()
    qry = """ select max(logical_time) from {}; """.format(edgesTableName)
    time = cur.execute(qry).fetchone()
    qry = """ update {} set time = ({})""".format(counterTableName, time[0])
    cur.execute(qry)
    conn.commit()
    global graphId
    qry = """ update {} set graph_id = ({})""".format(graphTableName, graphId+1)
    cur.execute(qry)
    conn.commit()
    
def update(line):
    global timeStart
    global graphId 
    line[5] = int(line[5]) + timeStart
    line.append(graphId)
    return(line)


In [9]:
def convertTxtToCsv(fname):
    with open( 'parsed-graphs/' + fname + '.txt', 'r') as in_file:
        stripped = (line.strip() for line in in_file)
        lines = (update(re.split(' |:', line)) for line in stripped if line)
        with open(fname + '.csv', 'w') as out_file:
            writer = csv.writer(out_file)
            writer.writerow(('src_id', 'dest_id', 'src_type', 'dest_type', 'edge_type', 'logical_time', 'graph_id'))
            writer.writerows(lines)

def populateDB(fname):
    # Connect to SQLite database
    conn = sqlite3.connect(r'toy.db')

    # Load CSV data into Pandas DataFrame
    stud_data = pd.read_csv(fname + '.csv')
    # Write the data to a sqlite table
    stud_data.to_sql(edgesTableName, conn, if_exists='append', index=False)

    conn.close()

    

In [10]:
   
### for training data
def train():
    fnames = []
    for i in range(0, 26):
        fnames.append('base-'+str(i))
    print(fnames)
    with tqdm(total= len(fnames)) as pbar1:
        for f in fnames:
            prepare()
            global timeStart
            print(timeStart)
            convertTxtToCsv(f)
            populateDB(f)
            updateDB()
            os.remove(f + '.csv')
            pbar1.update(1)

train()

['base-0', 'base-1', 'base-2', 'base-3', 'base-4', 'base-5', 'base-6', 'base-7', 'base-8', 'base-9', 'base-10', 'base-11', 'base-12', 'base-13', 'base-14', 'base-15', 'base-16', 'base-17', 'base-18', 'base-19', 'base-20', 'base-21', 'base-22', 'base-23', 'base-24', 'base-25']


  0%|          | 0/26 [00:00<?, ?it/s]

time - 0  id - 0
0
time - 104968  id - 1
104968
time - 181480  id - 2
181480
time - 392325  id - 3
392325
time - 471690  id - 4
471690
time - 571235  id - 5
571235
time - 681808  id - 6
681808
time - 760090  id - 7
760090
time - 856573  id - 8
856573
time - 936587  id - 9
936587
time - 1006584  id - 10
1006584
time - 1105999  id - 11
1105999
time - 1230033  id - 12
1230033
time - 1340011  id - 13
1340011
time - 1456541  id - 14
1456541
time - 1543727  id - 15
1543727
time - 1641798  id - 16
1641798
time - 1742205  id - 17
1742205
time - 1838952  id - 18
1838952
time - 1960053  id - 19
1960053
time - 2086127  id - 20
2086127
time - 2162683  id - 21
2162683
time - 2260271  id - 22
2260271
time - 2374342  id - 23
2374342
time - 2467239  id - 24
2467239
time - 2599882  id - 25
2599882
